# Assignment 3
## Exercise 1 + 2
### Import everything

In [9]:
from cifar_helper import CIFAR # note we renamed the helper file so it doesn't syntax error
%matplotlib notebook
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

### Create a CIFAR data loader
We are using the one that was provided. We had to import numpy there because it was missing as a dependency.

In [13]:
cifar = CIFAR("data")

## Exercise 3 - Investigating the data

In [14]:
# Get some data to inspect.
test_batch = list(zip(cifar._training_data[:16], cifar._training_labels[:16]))

# Define the categories that correspond to the numeric labels.
categories = ['Plane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse',
        'Ship', 'Truck']
f, axarr = plt.subplots(4, 4)
for i in range(4):
    for j in range(4):
        index = 4 * i + j
        ax = axarr[i][j]
        img = test_batch[index][0]
        label = test_batch[index][1]
        ax.set_title(categories[label])
        ax.imshow(img)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
plt.show()

## !!!TODO!!! Exercise 4
TODO: How many neurons are simulated? How many degrees of freedom (weights) does the network have? Of many floating- point operations are necessary for a forward pass of the network? (Write down your answer!)

## Exercise 5
Now we are implementing the DFG.

In [22]:
# Reset graph in order to cope with multiple cell executions.
tf.reset_default_graph()

# Define placeholders for both the input x and the labels l.
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input')
l = tf.placeholder(dtype=tf.uint8, shape=(None, 1), name='labels')
# The labels are encoded as one-hot vectors.
l_one_hot = tf.squeeze(tf.one_hot(l, 10), axis=1)


#############
# LAYER ONE #
#############

# There are 16 Kernels with a shape of 5x5x3 each, plus a bias.
kernel_shape1 = (5, 5, 3, 16)
bias_shape1   = (16,)

# Now we initialize the kernel with values from a truncated normal and the bias with zero.
kernels1 = tf.Variable(tf.truncated_normal(kernel_shape1, stddev=0.1, seed=10), kernel_shape1)
biases1  = tf.Variable(tf.zeros(bias_shape1), dtype=tf.float32)

# Here we are defining the convolution operation by applying kernel to the input, aka the images.
# Stride is (1,1,1,1) and padding is SAME since we want to keep the dimensions.
conv1 = tf.nn.conv2d(x, kernels1, strides=(1, 1, 1, 1), padding='SAME', name='conv1')

# As activation function we are using a hyperbolic tangent.
activation1 = tf.nn.tanh(conv1 + biases1, name='activation1')


#############
# LAYER TWO #
#############

# Now we are pooling the results with a 2x2 kernel in order to get 16 16x16 feature maps.
pool1 = tf.nn.max_pool(activation1, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding='SAME')


###############
# LAYER THREE #
###############

# Again we are defining the shapes of our new kernels.
# We have 16 16x16 feature maps and we want 32 16x16 feature maps based on a 32 3x3 kernels.
kernel_shape2 = (3, 3, 16, 32)
bias_shape2   = (32,)
kernels2      = tf.Variable(tf.truncated_normal(kernel_shape2, stddev=0.1, seed=10), kernel_shape2)
biases2       = tf.Variable(tf.zeros(bias_shape2), dtype=tf.float32)

# Define the next convolution operation with the previous output pool1 and the new kernels kernels2.
conv2         = tf.nn.conv2d(pool1, kernels2, strides=(1, 1, 1, 1), padding='SAME', name='conv2')
activation2   = tf.nn.tanh(conv2 + biases2, name='activation2')


##############
# LAYER FOUR #
##############

# Again we are pooling the results with a 2x2 kernel in order to get 32 8x8 feature maps.
pool2 = tf.nn.max_pool(activation2, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1), padding='SAME')


##############
# LAYER FIVE #
##############

# Now we want to direct the output into a FFNN. For that reason we have to reshape the data into
# a vector format.
pool2_reshaped = tf.reshape(pool2, (-1, 2048), name='reshaped1')

# There are 512 neurons that are connected to 8*8*32=2048 kernel pixels.
fc_weights1 = tf.get_variable(
    'fully-connected-1-weights',
    initializer=tf.truncated_normal_initializer(),
    shape=(8 * 8 * 32, 512),
    dtype=tf.float32
)

# Each neuron has a bias associated.
fc_bias1 = tf.get_variable(
    'fully-connected-1-bias', 
    initializer=tf.zeros_initializer(), 
    shape=(512,)
)

# The activation function.
fc1 = tf.nn.tanh(tf.matmul(pool2_reshaped, fc_weights1) + fc_bias1)


#############
# LAYER SIX #
#############

fc_weights2 = tf.get_variable(
        'fully-connected-2-weights',
        initializer=tf.truncated_normal_initializer(),
        shape=(512, 10),
        dtype=tf.float32
      )
fc_bias2 = tf.get_variable(
    'fully-connected-2-bias', 
    initializer=tf.zeros_initializer(), 
    shape=(10,)
)

# Now we compute the drive of each neuron.
fc2_logit = tf.matmul(fc1, fc_weights2) + fc_bias2

# for the last layer we are choosing a softmax activation function and, like in previous
# homeworks, reduce the cross entropy during training.
cross_entropy      = tf.nn.softmax_cross_entropy_with_logits(logits=fc2_logit, labels=l_one_hot)
mean_cross_entropy = tf.reduce_mean(cross_entropy)

## Exercise 6
Here we are defining the optimizer and the accuracy and actually train the network

In [21]:
# Set the training parameters.
batch_size    = 100
learning_rate = 1e-4

# Here we are using the AdamOptimizer as it yields better results in our task.
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_cross_entropy)

# Check whether the strongest firing neuron coincides with the max value position in the real labels.
correct_prediction = tf.equal(tf.argmax(fc2_logit, 1), tf.argmax(l_one_hot, 1))

# The accuracy is the mean of the matching results with the respective labels.
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

for index, (data, labels) in enumerate(cifar.get_training_batch(batch_size)):
    print(labels.shape)
return

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    epochs = 3
    for epoch in range(epochs):
        print('Starting epoch %d' % epoch)
        for index, (data, labels) in enumerate(cifar.get_training_batch(batch_size)):
            sess.run(
                [train_step],
                feed_dict={x: data, l: labels[:, np.newaxis]}
            )
            if index % 20 == 0:
                test_acc = sess.run([accuracy], feed_dict={x:
                    cifar._training_data[:1000], l: cifar._training_labels[:,
                        np.newaxis][:1000]})
                print('Current train accuracy %f' % test_acc[0])